In [ ]:
# Packages
!pip install -q pyngrok

# Libraries
import os
from google.colab import userdata
import time
from pyngrok import ngrok
from getpass import getpass

In [ ]:
# Clone repository (skip if already exists)
if not os.path.exists('CMPE259_Project'):
    !git clone https://github.com/bvegaloj/CMPE259_Project.git
else:
    print("Repository already cloned")

%cd CMPE259_Project

# Install dependencies
!pip install -q -r requirements.txt

# Install localtunnel for public URL (alternative to ngrok)
!npm install -g localtunnel

# Populate the database
print("\nSetting up database")
import sys
sys.path.insert(0, '/content/CMPE259_Project')

# Run populate_data.py
import subprocess
result = subprocess.run(
    ['python', 'data/populate_data.py'],
    cwd='/content/CMPE259_Project',
    env={**os.environ, 'PYTHONPATH': '/content/CMPE259_Project'},
    capture_output=True,
    text=True
)
print(result.stdout)
if result.returncode != 0:
    print("Error:", result.stderr)
else:
    print("Database populated")

In [ ]:
# Set API keys securely from Colab secrets
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
os.environ['NGROK_API_KEY'] = userdata.get('NGROK_API_KEY')

print("API keys set")

In [ ]:
# Kill any existing processes
print("Stopping existing processes")
!pkill -f streamlit
!pkill -f ngrok
time.sleep(3)

# Set your ngrok authtoken
ngrok_token = os.environ['NGROK_API_KEY']
!ngrok config add-authtoken {ngrok_token}

# Check if port 8501 is free, if not kill specifically
!fuser -k 8501/tcp

# Start Streamlit directly 
print("Starting Streamlit App")

# Run app
!streamlit run ui/streamlit_app.py --server.port 8501 --server.headless true > streamlit.log 2>&1 &

# Wait for startup
time.sleep(5)

# Check logs to ensure it started
print("Startup Logs:")
!tail -n 20 streamlit.log

# Establish tunnel
try:
    # Disconnect old tunnels
    ngrok.kill()

    # Connect new tunnel
    public_url = ngrok.connect(8501)
    print(f"Streamlit running at: {public_url}")
except Exception as e:
    print(f"\nNgrok error: {e}")